# EuropePMC accession number data descriptive stats

Publication -> data accession numbers are available from EuropePMC. These can be linked to the Data Citation Corpus using DOIs for publications and accession numbers (or DOIs in some cases) for datasets. See [README](README.md) for more details.

## Setup:

In [62]:
from pathlib import Path
import re
import json

In [63]:
from download_data import load_accession_data, accession_csv_to_dataframe, prepare_concatenated_dataframe, postprocess_europepmc_accession_data

In [64]:
import pandas as pd
import numpy as np

In [65]:
import logging
logging.getLogger('backoff').addHandler(logging.StreamHandler())
logging.getLogger('backoff').setLevel(logging.INFO)

In [66]:
def get_accession_dataframe(data_directory=None, postprocess=True):
    if data_directory is None:
        # default is to download the data from EuropePMC
        return load_accession_data(postprocess=postprocess)
    else:
        csv_files = Path(data_directory).glob("*.csv")
        _dfs = []
        for file in csv_files:
            _dfs.append(accession_csv_to_dataframe(file, file.name))
        df_acc = prepare_concatenated_dataframe(_dfs)
        if postprocess is True:
            df_acc = postprocess_europepmc_accession_data(df_acc)
        return df_acc
            

## Get EuropePMC data

We will get a dataframe where each row is a PMCID -> accession number pair.

We can either: (a) download the EuropePMC accession data manually, then specify the folder with all of the CSV files:

```py
df_acc = get_accession_dataframe("<path_to_folder>", postprocess=True)
```

or, (b) download the data on the fly:

```py
df_acc = get_accession_dataframe(postprocess=True)
```

The data will look something like:

```mermaid
erDiagram
    EUROPEPMC["EuropePMC Accession Number Citation"] {
        string accession_number PK
        string PMCID PK
        category repository_europepmc
    }
```


In [67]:
df_acc = get_accession_dataframe(postprocess=True)

Backing off make_request(...) for 0.7s (requests.exceptions.HTTPError: 502 Server Error: Proxy Error for url: https://europepmc.org/ftp/TextMinedTerms/hipsci.csv)
Backing off make_request(...) for 0.7s (requests.exceptions.HTTPError: 502 Server Error: Proxy Error for url: https://europepmc.org/ftp/TextMinedTerms/hipsci.csv)


In [68]:
df_acc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7871913 entries, 0 to 8174537
Data columns (total 5 columns):
 #   Column                Dtype   
---  ------                -----   
 0   accession_number      string  
 1   PMCID                 string  
 2   EXTID                 string  
 3   SOURCE                string  
 4   repository_europepmc  category
dtypes: category(1), string(4)
memory usage: 307.8 MB


## Get Corpus data

I'm using a preprocessed file I made from the downloaded Corpus data.

The data will look something like (some columns omitted):

```mermaid
erDiagram
    CORPUS["Corpus Citation"] {
        string id PK
        string publication_id "Currently only DOI allowed"
        string dataset_id "DOI or accession number"
        category repository "not necessarily matched to the EuropePMC repository"
        bool publication_is_doi
        bool dataset_is_doi
    }
```


In [69]:
cols = ['publisher', 'journal', 'repository', 'publication', 'dataset', 'publication_is_doi', 'dataset_is_doi', 'publishedDate', 'source']
df_corpus = pd.read_parquet('../data/df_2024-08-23-data-citation-corpus-v2.0.parquet', columns=cols)

In [70]:
df_corpus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5256114 entries, 61b9de16-8243-4802-b9d9-d04b7c46948e to 30ea1b0e-057f-4cfe-9566-080eaa184706
Data columns (total 9 columns):
 #   Column              Dtype              
---  ------              -----              
 0   publisher           category           
 1   journal             category           
 2   repository          category           
 3   publication         string             
 4   dataset             string             
 5   publication_is_doi  bool               
 6   dataset_is_doi      bool               
 7   publishedDate       datetime64[ns, UTC]
 8   source              category           
dtypes: bool(2), category(4), datetime64[ns, UTC](1), string(2)
memory usage: 206.3+ MB


## Get DOI data from EuropePMC

The data file from EuropePMC links DOI to one or both of PMID/PMCID. We will only use the data linking PMCID to DOI.

In [71]:
# Load DOI data (frome EuropePMC)
import sys
sys.path.append('..')
from clean_doi import clean_doi
fp = Path('../data/europepmc/PMID_PMCID_DOI.csv.gz')
df_pmid_doi = pd.read_csv(fp)
print(f"dataframe shape {df_pmid_doi.shape}")
df_pmid_doi = df_pmid_doi.dropna(subset=['PMCID', 'DOI'])
print(f"dataframe shape {df_pmid_doi.shape}")
df_pmid_doi['DOI'] = df_pmid_doi['DOI'].apply(clean_doi, return_none_if_error=True)
df_pmid_doi = df_pmid_doi.drop_duplicates(subset=['PMCID', 'DOI'])
print(f"dataframe shape {df_pmid_doi.shape}")

dataframe shape (39155263, 3)
dataframe shape (8849412, 3)
dataframe shape (8848632, 3)


In [72]:
df_acc = df_acc.merge(df_pmid_doi, how='left', on='PMCID')

In [73]:
df_acc['DOI'].isna().value_counts()

DOI
False    7736494
True      188480
Name: count, dtype: int64

In [74]:
# how many Corpus publication DOIs have PMCID?
corpus_dois = df_corpus[df_corpus['publication_is_doi']==True]['publication'].unique()
europepmc_dois = df_pmid_doi['DOI'].unique()
doi_intersection = set.intersection(set(corpus_dois), set(europepmc_dois))
print(f"There are {len(corpus_dois):,} publication DOIs in the Corpus.")
print(f"{len(doi_intersection):,} ({len(doi_intersection)/len(corpus_dois):.0%}) of these have PMCID associated.")

There are 2,024,203 publication DOIs in the Corpus.
1,420,535 (70%) of these have PMCID associated.


Desired descriptive stats table:

```mermaid
erDiagram
    REPOSITORY {
        category repository_europepmc PK
        int row_count_europepmc
    }
    REPCORPUS["Repository in Data Corpus"] {
        string corpus_repository PK
        int row_count
    }
    REPLINK["Repository names in Corpus linked to EuropePMC"] {
        category repository_europepmc PK, FK
        string corpus_repository PK, FK
        int row_count
    }
    REPOSITORY one to one or more REPLINK : is
    REPCORPUS one to zero or more REPLINK : matches
```

this will allow us to get:

```mermaid
erDiagram
    REPOSITORY {
        category repository_europepmc PK
        int row_count_europepmc
        int count_matched_in_corpus
        float pct_matched_in_corpus
        string[] names_in_corpus
    }
```

In [75]:
df_repcorpus = df_corpus[df_corpus['dataset_is_doi']==False]['repository'].value_counts().reset_index(name="row_count")
df_repcorpus = df_repcorpus[df_repcorpus["row_count"]>0]

In [76]:
df_repository = df_acc['repository_europepmc'].value_counts().reset_index(name="row_count_europepmc")

In [77]:
#upper?
acc_nums = df_acc['accession_number'].dropna()
# (acc_nums == acc_nums.str.upper()).all()
acc_nums[acc_nums!=acc_nums.str.upper()]

135950     1h3iA01
135951     1mt6A01
135952     1tcvA00
135953     1td1B00
135954     1fp5A02
            ...   
7924903     p10347
7924914     p00698
7924924     p47733
7924943     H2Bub1
7924968     B7Sym9
Name: accession_number, Length: 1188282, dtype: string

In [78]:
to_merge = df_corpus[['publication', 'dataset', 'repository']].reset_index()
to_merge = to_merge.dropna(subset=['publication', 'dataset'])
to_merge['accession_number_upper'] = to_merge['dataset'].str.upper()
to_merge = to_merge.drop(columns=['dataset'])
to_merge = to_merge.rename(columns={'publication': 'DOI', 'id': 'corpus_id', 'repository': 'corpus_repository'})
df_acc['accession_number_upper'] = df_acc['accession_number'].str.upper()
df_acc_corpus_merge = df_acc.merge(to_merge, how='left', on=['DOI', 'accession_number_upper'])

In [79]:
df_acc_match1 = df_acc_corpus_merge.dropna(subset=['corpus_id'])
print(f"matched {len(df_acc_match1):,} publication --> dataset citations between EuropePMC and Corpus, using exact matching on DOI and accession number")

matched 1,341,556 publication --> dataset citations between EuropePMC and Corpus, using exact matching on DOI and accession number


In [80]:
df_acc_match1

,accession_number,PMCID,EXTID,SOURCE,repository_europepmc,PMID,DOI,accession_number_upper,corpus_id,corpus_repository
1486,E-TABM-63,PMC2944327,19728874,MED,arrayexpress,19728874.0,10.1186/1752-0509-3-86,E-TABM-63,920ae419-48d6-44bf-8c1b-56e112df5576,ArrayExpress
1492,E-MEXP-449,PMC2944327,19728874,MED,arrayexpress,19728874.0,10.1186/1752-0509-3-86,E-MEXP-449,2ae1c77b-67c6-4075-a109-f8e4fff2557e,ArrayExpress
1494,E-MEXP-1094,PMC2944327,19728874,MED,arrayexpress,19728874.0,10.1186/1752-0509-3-86,E-MEXP-1094,ea61eb9e-0c17-47d7-8e54-eea1e8632607,ArrayExpress
1495,E-MEXP-547,PMC2944327,19728874,MED,arrayexpress,19728874.0,10.1186/1752-0509-3-86,E-MEXP-547,6f788fae-7808-4856-a30f-37d7a9cc3e48,ArrayExpress
1498,E-GEOD-431,PMC2944327,19728874,MED,arrayexpress,19728874.0,10.1186/1752-0509-3-86,E-GEOD-431,f2e36d1e-15fd-423b-b4be-84ad0e082db0,ArrayExpress
...,...,...,...,...,...,...,...,...,...,...
7929780,P53355,PMC2199731,10402466,MED,uniprot,10402466.0,10.1083/jcb.146.1.141,P53355,aae4869f-9ef6-4ccf-a23a-36d69ca7b401,UniProt
7929804,P11047,PMC2185082,10225960,MED,uniprot,10225960.0,10.1083/jcb.145.3.605,P11047,9629c023-674f-4474-b0cb-f97fcc85e3c5,UniProt
7929825,P42271,PMC2132928,10037793,MED,uniprot,10037793.0,10.1083/jcb.144.4.721,P42271,d50a7943-fa5e-4e18-8015-b6d49889bc67,UniProt
7929828,Q33211,PMC9649441,PMCPMC9649441,CTX,uniprot,36333500.0,10.1038/s41588-022-01210-z,Q33211,ca7ffcac-1cef-4280-98cf-12c70314757c,UniProt


In [81]:
df_acc_match1['corpus_repository'].drop_duplicates().tolist()

['ArrayExpress',
 'BioStudies',
 'BioProject',
 'European Nucleotide Archive',
 nan,
 'CATH',
 'ChEMBL',
 'Complex Portal (CP)',
 'NCBI dbGaP (DataBase of Genotypes And Phenotypesgenotypes and phenotypes)',
 'Harvard Dataverse',
 'GigaScience Database',
 'Dryad',
 'Zenodo',
 'The Global Biodiversity Information Facility',
 'PANGAEA',
 'NASA Ocean Biology Distributed Active Archive Center',
 'Mercator Ocean International',
 'NOAA National Centers for Environmental Information',
 'figshare',
 'Mendeley',
 'EBRAINS',
 'Human Brain Project Neuroinformatics Platform',
 'Portail Data INRAE',
 'F1000Research',
 'MX-RDR',
 'ECMWF',
 'CaltechDATA',
 'Ocean Biogeographic Information System',
 'Environmental Data Initiative',
 'VLIZ',
 'The Network Data Exchange - NDEx',
 'GBIF Secretariat',
 'MorphoBank',
 'University of Bath',
 'Earth System Grid Federation',
 'The Cancer Imaging Archive',
 'F1000 Research Limited',
 'FaceBase (www.facebase.org)',
 'Open Context',
 'CORA.Repositori de Dades de 

In [82]:
pivot = df_acc_match1.pivot_table(
    values=["corpus_id", "corpus_repository"],
    index="repository_europepmc",
    aggfunc={
        "corpus_id": len,
        "corpus_repository": lambda x: x.drop_duplicates().tolist(),
    },
    observed=True,
).reset_index().rename(columns={'corpus_id': 'count_matched_in_corpus'})

In [83]:
# create descriptive stats table
df_repository_stats = df_repository.merge(pivot, how='left', on='repository_europepmc')
df_repository_stats['count_matched_in_corpus'] = df_repository_stats['count_matched_in_corpus'].fillna(value=0).astype(int)
df_repository_stats['corpus_repository'] = df_repository_stats['corpus_repository'].fillna(value="")
df_repository_stats['pct_matched_in_corpus'] = df_repository_stats['count_matched_in_corpus'] / df_repository_stats['row_count_europepmc']
df_repository_stats = df_repository_stats.rename(columns={'corpus_repository': 'names_in_corpus'})
df_repository_stats = df_repository_stats.set_index('repository_europepmc', verify_integrity=True).sort_index()
# rearrange columns
column_order = ['row_count_europepmc', 'count_matched_in_corpus', 'pct_matched_in_corpus', 'names_in_corpus']
df_repository_stats = df_repository_stats[column_order]

In [84]:
df_repository_stats

,row_count_europepmc,count_matched_in_corpus,pct_matched_in_corpus,names_in_corpus
repository_europepmc,,,,
alphafold,1316,0,0.000000,
arrayexpress,14594,7493,0.513430,[ArrayExpress]
bia,91,19,0.208791,[BioStudies]
biomodels,839,0,0.000000,
bioproject,92712,46059,0.496797,"[BioProject, European Nucleotide Archive]"
biosample,18365,5128,0.279227,"[nan, European Nucleotide Archive]"
biostudies,268,106,0.395522,[BioStudies]
brenda,7347,0,0.000000,
cath,1745,456,0.261318,[CATH]


In [24]:
df_repository_stats.to_clipboard()

In [85]:
df_uniprot = df_acc_corpus_merge[df_acc_corpus_merge["repository_europepmc"]=="uniprot"]
df_uniprot.shape

(608571, 10)

In [86]:
df_uniprot_corpusmatch = df_uniprot[df_uniprot["corpus_id"].notna()]
df_uniprot_corpusmatch.shape

(93156, 10)

In [87]:
# sample by doi
random_seed = 113
_unique_dois = df_uniprot_corpusmatch["DOI"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_uniprot_corpusmatch_sample = df_uniprot[df_uniprot["DOI"].isin(_sample_dois)].sort_values("DOI")
df_uniprot_corpusmatch_sample["found_in_corpus"] = df_uniprot_corpusmatch_sample["corpus_id"].notna()
df_uniprot_corpusmatch_sample["doi_index"] = df_uniprot_corpusmatch_sample.groupby("DOI").ngroup()

In [88]:
df_uniprot_corpusmatch_sample.to_clipboard(index=False)

In [89]:
df_uniprot_pubnotincorpus = df_uniprot[~(df_uniprot["DOI"].isin(corpus_dois))]
df_uniprot_pubnotincorpus.shape

(280817, 10)

In [90]:
df_uniprot_pubnotincorpus["DOI"].nunique()

33162

In [91]:
_unique_dois = df_uniprot_pubnotincorpus["DOI"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_uniprot_pubnotincorpus_sample = df_uniprot[df_uniprot["DOI"].isin(_sample_dois)].sort_values("DOI")
df_uniprot_pubnotincorpus_sample["found_in_corpus"] = df_uniprot_pubnotincorpus_sample["corpus_id"].notna()
df_uniprot_pubnotincorpus_sample["doi_index"] = df_uniprot_pubnotincorpus_sample.groupby("DOI").ngroup()

In [92]:
column_order = """doi_index	DOI	accession_number	found_in_corpus	PMCID	EXTID	SOURCE	repository_europepmc	PMID	accession_number_upper	corpus_id	corpus_repository""".split()
df_uniprot_pubnotincorpus_sample[column_order].to_clipboard(index=False)

In [93]:
matched_corpus_ids = df_acc_corpus_merge["corpus_id"].dropna().unique()
df_corpus_unmatched = df_corpus[~(df_corpus.index.isin(matched_corpus_ids))]

In [94]:
df_corpus_unmatched_uniprot = df_corpus_unmatched[df_corpus_unmatched["repository"].str.lower()=="uniprot"]
df_corpus_unmatched_uniprot.shape

(66169, 9)

In [95]:
_unique_dois = df_corpus_unmatched_uniprot["publication"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_corpus_unmatched_uniprot_sample = df_corpus_unmatched_uniprot[df_corpus_unmatched_uniprot["publication"].isin(_sample_dois)].sort_values("publication")
df_corpus_unmatched_uniprot_sample["found_in_corpus"] = df_corpus_unmatched_uniprot_sample.index.isin(df_acc_corpus_merge["corpus_id"].dropna().unique())
df_corpus_unmatched_uniprot_sample["doi_index"] = df_corpus_unmatched_uniprot_sample.groupby("publication").ngroup()

In [58]:
df_corpus_unmatched_uniprot_sample.to_clipboard()

### Protein Data Bank

In [96]:
df_pdb = df_acc_corpus_merge[df_acc_corpus_merge["repository_europepmc"]=="pdb"]
df_pdb.shape

(788008, 10)

In [97]:
df_pdb_corpusmatch = df_pdb[df_pdb["corpus_id"].notna()]
df_pdb_corpusmatch.shape

(221048, 10)

In [98]:
# sample by doi
random_seed = 113
_unique_dois = df_pdb_corpusmatch["DOI"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_pdb_corpusmatch_sample = df_pdb[df_pdb["DOI"].isin(_sample_dois)].sort_values("DOI")
df_pdb_corpusmatch_sample["found_in_corpus"] = df_pdb_corpusmatch_sample["corpus_id"].notna()
df_pdb_corpusmatch_sample["doi_index"] = df_pdb_corpusmatch_sample.groupby("DOI").ngroup()

In [ ]:
df_pdb_corpusmatch_sample.to_clipboard(index=False)

In [99]:
df_pdb_pubnotincorpus = df_pdb[~(df_pdb["DOI"].isin(corpus_dois))]
df_pdb_pubnotincorpus.shape

(454155, 10)

In [100]:
df_pdb_pubnotincorpus["DOI"].nunique()

122291

In [101]:
_unique_dois = df_pdb_pubnotincorpus["DOI"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_pdb_pubnotincorpus_sample = df_pdb[df_pdb["DOI"].isin(_sample_dois)].sort_values("DOI")
df_pdb_pubnotincorpus_sample["found_in_corpus"] = df_pdb_pubnotincorpus_sample["corpus_id"].notna()
df_pdb_pubnotincorpus_sample["doi_index"] = df_pdb_pubnotincorpus_sample.groupby("DOI").ngroup()

In [ ]:
column_order = """doi_index	DOI	accession_number	found_in_corpus	PMCID	EXTID	SOURCE	repository_europepmc	PMID	accession_number_upper	corpus_id	corpus_repository""".split()
df_pdb_pubnotincorpus_sample[column_order].to_clipboard(index=False)

In [102]:
matched_corpus_ids = df_acc_corpus_merge["corpus_id"].dropna().unique()
df_corpus_unmatched = df_corpus[~(df_corpus.index.isin(matched_corpus_ids))]

In [120]:
df_corpus_unmatched_pdb = df_corpus_unmatched[df_corpus_unmatched["repository"]=="The Protein Data Bank"]
df_corpus_unmatched_pdb.shape

(184986, 9)

In [ ]:
_unique_dois = df_corpus_unmatched_pdb["publication"].drop_duplicates()
_sample_dois = _unique_dois.sample(n=50, random_state=random_seed)
df_corpus_unmatched_pdb_sample = df_corpus_unmatched_pdb[df_corpus_unmatched_pdb["publication"].isin(_sample_dois)].sort_values("publication")
df_corpus_unmatched_pdb_sample["found_in_corpus"] = df_corpus_unmatched_pdb_sample.index.isin(df_acc_corpus_merge["corpus_id"].dropna().unique())
df_corpus_unmatched_pdb_sample["doi_index"] = df_corpus_unmatched_pdb_sample.groupby("publication").ngroup()

In [ ]:
df_corpus_unmatched_pdb_sample.to_clipboard()